In [5]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
no_of_sample = 10
#CUDA = True
BATCH_SIZE = 32
LOG_INTERVAL = 5

In [4]:
torch.cuda.empty_cache()

In [6]:
class CelebaDataset(Dataset):

    def __init__(self, root_dir, im_name_list, resize_dim, transform=None):
        self.root_dir = root_dir
        self.im_list = im_name_list
        self.resize_dim = resize_dim
        self.transform = transform

    def __len__(self):
        return len(self.im_list)

    def __getitem__(self, idx):
        im = Image.open(os.path.join(self.root_dir, self.im_list[idx]))
        im = np.array(im)
        im = im[...,:3]
        im = imresize(im, self.resize_dim, interp='nearest')
        im = im / 255

        if self.transform:
            im = self.transform(im)

        return im

class ToTensor(object):
    """Convert ndarrays in sample to Tensors. numpy image: H x W x C, torch image: C X H X W
    """

    def __call__(self, image, invert_arrays=True):

        if invert_arrays:
            image = image.transpose((2, 0, 1))

        return torch.from_numpy(image)
    


In [7]:
class Conv_Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride, pool_kernel_size=(2, 2)):
        super(Conv_Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding, stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding, stride)
        self.pool = nn.MaxPool2d(pool_kernel_size)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = self.pool(x)

        return x
    
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # Encoder
        self.block1 = Conv_Block(3, 64, (3, 3), 1, 1)  # 64
        self.block2 = Conv_Block(64, 128, (3, 3), 1, 1)  # 32
        self.block3 = Conv_Block(128, 256, (3, 3), 1, 1)  # 16
        self.block4 = Conv_Block(256, 32, (3, 3), 1, 1)  # 8

        # Decoder
        self.fct_decode = nn.Sequential(
            nn.Conv2d(16, 64, (3, 3), padding=1),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode='nearest'),  # 16
            nn.Conv2d(64, 64, (3, 3), padding=1),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode='nearest'),  # 32
            nn.Conv2d(64, 64, (3, 3), padding=1),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode='nearest'),  # 64
            nn.Conv2d(64, 16, (3, 3), padding=1),
            nn.ELU(),
            nn.Upsample(scale_factor=2, mode='nearest'),  # 128
        )

        self.final_decod_mean = nn.Conv2d(16, 3, (3, 3), padding=1)

    def encode(self, x):
        '''return mu_z and logvar_z'''

        x = F.elu(self.block1(x))
        x = F.elu(self.block2(x))
        x = F.elu(self.block3(x))
        x = F.elu(self.block4(x))

        return x[:, :16, :, :], x[:, 16:, :, :]  # output shape - batch_size x 16 x 8 x 8

    def reparameterize(self, mu: Variable, logvar: Variable) -> Variable:

        if self.training:
            # multiply log variance with 0.5, then in-place exponent
            # yielding the standard deviation

            sample_z = []
            for _ in range(no_of_sample):
                std = logvar.mul(0.5).exp_()  # type: Variable
                eps = Variable(std.data.new(std.size()).normal_())
                sample_z.append(eps.mul(std).add_(mu))

            return sample_z

        else:
            return mu

    def decode(self, z):

        z = self.fct_decode(z)
        z = self.final_decod_mean(z)
        z = F.sigmoid(z)

        return z.view(-1, 3 * 128 * 128)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        if self.training:
            return [self.decode(z) for z in z], mu, logvar
        else:
            return self.decode(z), mu, logvar

    def loss_function(self, recon_x, x, mu, logvar) -> Variable:
        # how well do input x and output recon_x agree?

        if self.training:
            BCE = 0
            for recon_x_one in recon_x:
                BCE += F.binary_cross_entropy(recon_x_one, x.view(-1, 3 * 128 * 128))
            BCE /= len(recon_x)
        else:
            BCE = F.binary_cross_entropy(recon_x, x.view(-1, 3 * 128 * 128))

        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        KLD /= BATCH_SIZE * 3 * 128 * 128

        return BCE + KLD

In [16]:
def train(epoch, model, optimizer, train_loader):
    # toggle model to train mode
    model.train()
    train_loss = 0
    # in the case of MNIST, len(train_loader.dataset) is 60000
    # each `data` is of BATCH_SIZE samples and has shape [128, 1, 28, 28]
    for batch_idx, data in enumerate(train_loader):
        data = Variable(data.type(torch.FloatTensor))
        data = data.cuda()
        optimizer.zero_grad()

        # push whole batch of data through VAE.forward() to get recon_loss
        recon_batch, mu, logvar = model(data)
        # calculate scalar loss
        loss = model.loss_function(recon_batch, data, mu, logvar)
        # calculate the gradient of the loss w.r.t. the graph leaves
        # i.e. input variables -- by the power of pytorch!
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [17]:
root_dir = "/home/ddmnn3/PycharmProjects/ADOS/Data/clip_images/all/"
image_files = os.listdir(root_dir)
train_dataset = CelebaDataset(root_dir, image_files[:20000], (128, 128), transforms.Compose([ToTensor()]))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=10, shuffle=True)

    

In [18]:
EPOCHS = 500
model = VAE().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [19]:
for epoch in range(1, EPOCHS + 1):
        train(epoch, model, optimizer, train_loader)
        #test(epoch, model, test_loader)

        torch.save(model.state_dict(), "/home/ddmnn3/PycharmProjects/ADOS/Models/" + "model_2")
        

Train Epoch: 1 [0/20000 (0%)]	Loss: 0.021788
Train Epoch: 1 [160/20000 (1%)]	Loss: 0.017835
Train Epoch: 1 [320/20000 (2%)]	Loss: 0.017255
Train Epoch: 1 [480/20000 (2%)]	Loss: 0.016716
Train Epoch: 1 [640/20000 (3%)]	Loss: 0.016490
Train Epoch: 1 [800/20000 (4%)]	Loss: 0.016210
Train Epoch: 1 [960/20000 (5%)]	Loss: 0.016050
Train Epoch: 1 [1120/20000 (6%)]	Loss: 0.016249
Train Epoch: 1 [1280/20000 (6%)]	Loss: 0.016242
Train Epoch: 1 [1440/20000 (7%)]	Loss: 0.016145
Train Epoch: 1 [1600/20000 (8%)]	Loss: 0.016046
Train Epoch: 1 [1760/20000 (9%)]	Loss: 0.016035
Train Epoch: 1 [1920/20000 (10%)]	Loss: 0.016191
Train Epoch: 1 [2080/20000 (10%)]	Loss: 0.017213
Train Epoch: 1 [2240/20000 (11%)]	Loss: 0.016853
Train Epoch: 1 [2400/20000 (12%)]	Loss: 0.016235
Train Epoch: 1 [2560/20000 (13%)]	Loss: 0.016269
Train Epoch: 1 [2720/20000 (14%)]	Loss: 0.016336
Train Epoch: 1 [2880/20000 (14%)]	Loss: 0.016093
Train Epoch: 1 [3040/20000 (15%)]	Loss: 0.016298
Train Epoch: 1 [3200/20000 (16%)]	Loss: 0

Train Epoch: 2 [6720/20000 (34%)]	Loss: 0.015903
Train Epoch: 2 [6880/20000 (34%)]	Loss: 0.015861
Train Epoch: 2 [7040/20000 (35%)]	Loss: 0.015576
Train Epoch: 2 [7200/20000 (36%)]	Loss: 0.015970
Train Epoch: 2 [7360/20000 (37%)]	Loss: 0.015769
Train Epoch: 2 [7520/20000 (38%)]	Loss: 0.015887
Train Epoch: 2 [7680/20000 (38%)]	Loss: 0.015771
Train Epoch: 2 [7840/20000 (39%)]	Loss: 0.015801
Train Epoch: 2 [8000/20000 (40%)]	Loss: 0.015648
Train Epoch: 2 [8160/20000 (41%)]	Loss: 0.016077
Train Epoch: 2 [8320/20000 (42%)]	Loss: 0.016135
Train Epoch: 2 [8480/20000 (42%)]	Loss: 0.015881
Train Epoch: 2 [8640/20000 (43%)]	Loss: 0.015999
Train Epoch: 2 [8800/20000 (44%)]	Loss: 0.015973
Train Epoch: 2 [8960/20000 (45%)]	Loss: 0.015955
Train Epoch: 2 [9120/20000 (46%)]	Loss: 0.015980
Train Epoch: 2 [9280/20000 (46%)]	Loss: 0.015886
Train Epoch: 2 [9440/20000 (47%)]	Loss: 0.016164
Train Epoch: 2 [9600/20000 (48%)]	Loss: 0.015694
Train Epoch: 2 [9760/20000 (49%)]	Loss: 0.015788
Train Epoch: 2 [9920

Train Epoch: 3 [13280/20000 (66%)]	Loss: 0.015860
Train Epoch: 3 [13440/20000 (67%)]	Loss: 0.015605
Train Epoch: 3 [13600/20000 (68%)]	Loss: 0.015765
Train Epoch: 3 [13760/20000 (69%)]	Loss: 0.015751
Train Epoch: 3 [13920/20000 (70%)]	Loss: 0.015947
Train Epoch: 3 [14080/20000 (70%)]	Loss: 0.015799
Train Epoch: 3 [14240/20000 (71%)]	Loss: 0.015849
Train Epoch: 3 [14400/20000 (72%)]	Loss: 0.015677
Train Epoch: 3 [14560/20000 (73%)]	Loss: 0.015773
Train Epoch: 3 [14720/20000 (74%)]	Loss: 0.015880
Train Epoch: 3 [14880/20000 (74%)]	Loss: 0.015814
Train Epoch: 3 [15040/20000 (75%)]	Loss: 0.015852
Train Epoch: 3 [15200/20000 (76%)]	Loss: 0.015744
Train Epoch: 3 [15360/20000 (77%)]	Loss: 0.015915
Train Epoch: 3 [15520/20000 (78%)]	Loss: 0.015813
Train Epoch: 3 [15680/20000 (78%)]	Loss: 0.015778
Train Epoch: 3 [15840/20000 (79%)]	Loss: 0.015684
Train Epoch: 3 [16000/20000 (80%)]	Loss: 0.015928
Train Epoch: 3 [16160/20000 (81%)]	Loss: 0.015973
Train Epoch: 3 [16320/20000 (82%)]	Loss: 0.015973


Train Epoch: 4 [19680/20000 (98%)]	Loss: 0.015939
Train Epoch: 4 [19840/20000 (99%)]	Loss: 0.015795
====> Epoch: 4 Average loss: 0.0158
Train Epoch: 5 [0/20000 (0%)]	Loss: 0.016109
Train Epoch: 5 [160/20000 (1%)]	Loss: 0.015627
Train Epoch: 5 [320/20000 (2%)]	Loss: 0.015816
Train Epoch: 5 [480/20000 (2%)]	Loss: 0.015687
Train Epoch: 5 [640/20000 (3%)]	Loss: 0.015918
Train Epoch: 5 [800/20000 (4%)]	Loss: 0.016002
Train Epoch: 5 [960/20000 (5%)]	Loss: 0.015933
Train Epoch: 5 [1120/20000 (6%)]	Loss: 0.015759
Train Epoch: 5 [1280/20000 (6%)]	Loss: 0.015645
Train Epoch: 5 [1440/20000 (7%)]	Loss: 0.015772
Train Epoch: 5 [1600/20000 (8%)]	Loss: 0.015843
Train Epoch: 5 [1760/20000 (9%)]	Loss: 0.015578
Train Epoch: 5 [1920/20000 (10%)]	Loss: 0.015874
Train Epoch: 5 [2080/20000 (10%)]	Loss: 0.015776
Train Epoch: 5 [2240/20000 (11%)]	Loss: 0.015783
Train Epoch: 5 [2400/20000 (12%)]	Loss: 0.015799
Train Epoch: 5 [2560/20000 (13%)]	Loss: 0.015724
Train Epoch: 5 [2720/20000 (14%)]	Loss: 0.015781
Tra

Train Epoch: 6 [6240/20000 (31%)]	Loss: 0.015897
Train Epoch: 6 [6400/20000 (32%)]	Loss: 0.015799
Train Epoch: 6 [6560/20000 (33%)]	Loss: 0.015784
Train Epoch: 6 [6720/20000 (34%)]	Loss: 0.015851
Train Epoch: 6 [6880/20000 (34%)]	Loss: 0.015645
Train Epoch: 6 [7040/20000 (35%)]	Loss: 0.015644
Train Epoch: 6 [7200/20000 (36%)]	Loss: 0.015759
Train Epoch: 6 [7360/20000 (37%)]	Loss: 0.015765
Train Epoch: 6 [7520/20000 (38%)]	Loss: 0.015993
Train Epoch: 6 [7680/20000 (38%)]	Loss: 0.015966
Train Epoch: 6 [7840/20000 (39%)]	Loss: 0.015967
Train Epoch: 6 [8000/20000 (40%)]	Loss: 0.015782
Train Epoch: 6 [8160/20000 (41%)]	Loss: 0.015863
Train Epoch: 6 [8320/20000 (42%)]	Loss: 0.015849
Train Epoch: 6 [8480/20000 (42%)]	Loss: 0.015849
Train Epoch: 6 [8640/20000 (43%)]	Loss: 0.015759
Train Epoch: 6 [8800/20000 (44%)]	Loss: 0.015657
Train Epoch: 6 [8960/20000 (45%)]	Loss: 0.015858
Train Epoch: 6 [9120/20000 (46%)]	Loss: 0.015657
Train Epoch: 6 [9280/20000 (46%)]	Loss: 0.015780
Train Epoch: 6 [9440

...

Train Epoch: 494 [14560/20000 (73%)]	Loss: 0.015754
Train Epoch: 494 [14720/20000 (74%)]	Loss: 0.015703
Train Epoch: 494 [14880/20000 (74%)]	Loss: 0.016169
Train Epoch: 494 [15040/20000 (75%)]	Loss: 0.015840
Train Epoch: 494 [15200/20000 (76%)]	Loss: 0.015718
Train Epoch: 494 [15360/20000 (77%)]	Loss: 0.015646
Train Epoch: 494 [15520/20000 (78%)]	Loss: 0.015873
Train Epoch: 494 [15680/20000 (78%)]	Loss: 0.015749
Train Epoch: 494 [15840/20000 (79%)]	Loss: 0.015801
Train Epoch: 494 [16000/20000 (80%)]	Loss: 0.015750
Train Epoch: 494 [16160/20000 (81%)]	Loss: 0.015857
Train Epoch: 494 [16320/20000 (82%)]	Loss: 0.015761
Train Epoch: 494 [16480/20000 (82%)]	Loss: 0.015738
Train Epoch: 494 [16640/20000 (83%)]	Loss: 0.015958
Train Epoch: 494 [16800/20000 (84%)]	Loss: 0.015842
Train Epoch: 494 [16960/20000 (85%)]	Loss: 0.015707
Train Epoch: 494 [17120/20000 (86%)]	Loss: 0.015749
Train Epoch: 494 [17280/20000 (86%)]	Loss: 0.015747
Train Epoch: 494 [17440/20000 (87%)]	Loss: 0.015907
Train Epoch:

====> Epoch: 495 Average loss: 0.0158
Train Epoch: 496 [0/20000 (0%)]	Loss: 0.016003
Train Epoch: 496 [160/20000 (1%)]	Loss: 0.015777
Train Epoch: 496 [320/20000 (2%)]	Loss: 0.015724
Train Epoch: 496 [480/20000 (2%)]	Loss: 0.015760
Train Epoch: 496 [640/20000 (3%)]	Loss: 0.015780
Train Epoch: 496 [800/20000 (4%)]	Loss: 0.015788
Train Epoch: 496 [960/20000 (5%)]	Loss: 0.015912
Train Epoch: 496 [1120/20000 (6%)]	Loss: 0.015698
Train Epoch: 496 [1280/20000 (6%)]	Loss: 0.015683
Train Epoch: 496 [1440/20000 (7%)]	Loss: 0.015785
Train Epoch: 496 [1600/20000 (8%)]	Loss: 0.015826
Train Epoch: 496 [1760/20000 (9%)]	Loss: 0.015701
Train Epoch: 496 [1920/20000 (10%)]	Loss: 0.015646
Train Epoch: 496 [2080/20000 (10%)]	Loss: 0.015677
Train Epoch: 496 [2240/20000 (11%)]	Loss: 0.015679
Train Epoch: 496 [2400/20000 (12%)]	Loss: 0.015800
Train Epoch: 496 [2560/20000 (13%)]	Loss: 0.015627
Train Epoch: 496 [2720/20000 (14%)]	Loss: 0.015684
Train Epoch: 496 [2880/20000 (14%)]	Loss: 0.015561
Train Epoch: 4

Train Epoch: 497 [5440/20000 (27%)]	Loss: 0.015814
Train Epoch: 497 [5600/20000 (28%)]	Loss: 0.015722
Train Epoch: 497 [5760/20000 (29%)]	Loss: 0.015795
Train Epoch: 497 [5920/20000 (30%)]	Loss: 0.015751
Train Epoch: 497 [6080/20000 (30%)]	Loss: 0.016010
Train Epoch: 497 [6240/20000 (31%)]	Loss: 0.015968
Train Epoch: 497 [6400/20000 (32%)]	Loss: 0.015526
Train Epoch: 497 [6560/20000 (33%)]	Loss: 0.015580
Train Epoch: 497 [6720/20000 (34%)]	Loss: 0.015738
Train Epoch: 497 [6880/20000 (34%)]	Loss: 0.015541
Train Epoch: 497 [7040/20000 (35%)]	Loss: 0.015729
Train Epoch: 497 [7200/20000 (36%)]	Loss: 0.015913
Train Epoch: 497 [7360/20000 (37%)]	Loss: 0.016113
Train Epoch: 497 [7520/20000 (38%)]	Loss: 0.015625
Train Epoch: 497 [7680/20000 (38%)]	Loss: 0.015635
Train Epoch: 497 [7840/20000 (39%)]	Loss: 0.016022
Train Epoch: 497 [8000/20000 (40%)]	Loss: 0.015732
Train Epoch: 497 [8160/20000 (41%)]	Loss: 0.015679
Train Epoch: 497 [8320/20000 (42%)]	Loss: 0.015748
Train Epoch: 497 [8480/20000 (4

Train Epoch: 498 [10880/20000 (54%)]	Loss: 0.015728
Train Epoch: 498 [11040/20000 (55%)]	Loss: 0.015876
Train Epoch: 498 [11200/20000 (56%)]	Loss: 0.015677
Train Epoch: 498 [11360/20000 (57%)]	Loss: 0.015608
Train Epoch: 498 [11520/20000 (58%)]	Loss: 0.015588
Train Epoch: 498 [11680/20000 (58%)]	Loss: 0.015662
Train Epoch: 498 [11840/20000 (59%)]	Loss: 0.015939
Train Epoch: 498 [12000/20000 (60%)]	Loss: 0.015868
Train Epoch: 498 [12160/20000 (61%)]	Loss: 0.015729
Train Epoch: 498 [12320/20000 (62%)]	Loss: 0.015667
Train Epoch: 498 [12480/20000 (62%)]	Loss: 0.015700
Train Epoch: 498 [12640/20000 (63%)]	Loss: 0.015709
Train Epoch: 498 [12800/20000 (64%)]	Loss: 0.015776
Train Epoch: 498 [12960/20000 (65%)]	Loss: 0.015618
Train Epoch: 498 [13120/20000 (66%)]	Loss: 0.015991
Train Epoch: 498 [13280/20000 (66%)]	Loss: 0.015876
Train Epoch: 498 [13440/20000 (67%)]	Loss: 0.015688
Train Epoch: 498 [13600/20000 (68%)]	Loss: 0.015571
Train Epoch: 498 [13760/20000 (69%)]	Loss: 0.015779
Train Epoch:

Train Epoch: 499 [16320/20000 (82%)]	Loss: 0.015608
Train Epoch: 499 [16480/20000 (82%)]	Loss: 0.015771
Train Epoch: 499 [16640/20000 (83%)]	Loss: 0.015686
Train Epoch: 499 [16800/20000 (84%)]	Loss: 0.015630
Train Epoch: 499 [16960/20000 (85%)]	Loss: 0.015707
Train Epoch: 499 [17120/20000 (86%)]	Loss: 0.015830
Train Epoch: 499 [17280/20000 (86%)]	Loss: 0.015776
Train Epoch: 499 [17440/20000 (87%)]	Loss: 0.015687
Train Epoch: 499 [17600/20000 (88%)]	Loss: 0.015636
Train Epoch: 499 [17760/20000 (89%)]	Loss: 0.016011
Train Epoch: 499 [17920/20000 (90%)]	Loss: 0.015774
Train Epoch: 499 [18080/20000 (90%)]	Loss: 0.015636
Train Epoch: 499 [18240/20000 (91%)]	Loss: 0.015657
Train Epoch: 499 [18400/20000 (92%)]	Loss: 0.015861
Train Epoch: 499 [18560/20000 (93%)]	Loss: 0.015752
Train Epoch: 499 [18720/20000 (94%)]	Loss: 0.015738
Train Epoch: 499 [18880/20000 (94%)]	Loss: 0.015921
Train Epoch: 499 [19040/20000 (95%)]	Loss: 0.015772
Train Epoch: 499 [19200/20000 (96%)]	Loss: 0.015734
Train Epoch:

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined